In [1]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_column', 500)

In [2]:
df = pd.read_csv("geodata.csv" , index_col=0)
mathpop = pd.read_csv("math.csv", index_col=0)
engpop = pd.read_csv("ela.csv", index_col=0)
#reading in data

In [3]:
df.head()

,District Code,Local Educational Agency,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds,STATEFP,UNSDLEA,GEOID,NAME,LSAD,LOGRADE,HIGRADE,MTFCC,SDTYP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,origin,ELSDLEA,SCSDLEA,treatment,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,per_capita_funds
0,61119,Alameda Unified,0.3486,"75,938,983","5,294,466",-,82158116,1611190000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,11031,2953,3306,0.299701,0.267700,7447.930015
1,61127,Albany City Unified,0.2833,"29,644,972","1,679,684",-,31352540,1611270000000,6.0,1860.0,601860.0,Albany City Unified,0.0,KG,12.0,G5420,NaN,E,4634305.0,9523833.0,37.890650,-122.318116,"POLYGON ((-122.373121 37.88388399999999, -122....",unified,NaN,NaN,0,3612,647,742,0.205426,0.179125,8680.105205
2,61143,Berkeley Unified,0.3519,"78,556,532","5,528,808",-,89369621,1611430000000,6.0,4740.0,604740.0,Berkeley Unified,0.0,KG,12.0,G5420,NaN,E,27095285.0,18715614.0,37.866972,-122.299147,"POLYGON ((-122.367781 37.866726, -122.356714 3...",unified,NaN,NaN,0,10022,2758,3153,0.314608,0.275195,8917.343943
3,61150,Castro Valley Unified,0.2904,"74,911,585","4,350,864",-,79554607,1611500000000,6.0,7800.0,607800.0,Castro Valley Unified,0.0,KG,12.0,G5420,NaN,E,177385526.0,4276387.0,37.719672,-122.046345,"POLYGON ((-122.179157 37.804981, -122.179139 3...",unified,NaN,NaN,0,9106,1811,2178,0.239183,0.198880,8736.504173
4,61168,Emery Unified,0.7953,"5,702,229","906,997","699,378",7326875,1611680000000,6.0,12630.0,612630.0,Emery Unified,0.0,KG,12.0,G5420,NaN,E,3317306.0,2517858.0,37.839906,-122.298963,"POLYGON ((-122.330139 37.841112, -122.324316 3...",unified,NaN,NaN,1,727,515,583,0.801926,0.708391,10078.232462


In [4]:
df.drop('LSAD', axis=1, inplace=True)
df.drop('LOGRADE', axis=1, inplace=True)
df.drop('HIGRADE', axis=1, inplace=True)
df.drop('MTFCC', axis=1, inplace=True)
df.drop('SDTYP', axis=1, inplace=True)
df.drop('FUNCSTAT', axis=1, inplace=True)
df.drop('ELSDLEA', axis=1, inplace=True)
df.drop('SCSDLEA', axis=1, inplace=True)
df.drop('origin', axis=1, inplace=True)
# dropping excess data
df = df.rename(columns={'NAME': 'districtname'})

In [5]:
df = pd.merge(df, mathpop, on=['districtname'], how='left')
# will need to merge separate dataframe for english scores
df = df.rename(columns={'county_name_x': 'county_name'})

One issue here is that I've had trouble merging both english and math scores onto one row of data corresponding to a specific student group within a district. I've tried left, right, inner, and outer merges. 

I'd like the dataframe to be structured like so:

- County
    - District
        - Student Group
            - English Score
            - Math Score
            
Instead, the student group gets duplicated for both sets of scores. 

- County
    - District
        - Student Group
            - English score
        - Student Group
            - Math score
            
I got around this for the capstone by simply duplicating the analysis for a second dataframe.

In [6]:
df.drop('District Code_x', axis=1, inplace=True)
df.drop('Local Educational Agency', axis=1, inplace=True)
# dropping excess data
df = df.rename(columns={'District Code_y': 'district_code'})
col_name = 'districtname'
first_col = df.pop(col_name)
df.insert(0, col_name, first_col)
col_2 = 'countyname'
second_col = df.pop(col_2)
df.insert(0, col_2, second_col)
#cleaning excess data, moving columns

In [7]:
math_df = df
math_df

,countyname,districtname,unduplicated_pupil_count,base_grant,supplemental_grant,concentration_grant,total_grants,cds,STATEFP,UNSDLEA,GEOID,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,treatment,enrollment_by_age,free_count_by_age,frpm_count_by_age,pct_frpm_by_age,pct_free_by_age,per_capita_funds,district_code,studentgroup,m_currstatus,m_priorstatus,m_change
0,NaN,NaN,0.3486,"75,938,983","5,294,466",-,82158116,1611190000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,11031,2953,3306,0.299701,0.267700,7447.930015,NaN,NaN,NaN,NaN,NaN
1,Alameda,Albany City Unified,0.2833,"29,644,972","1,679,684",-,31352540,1611270000000,6.0,1860.0,601860.0,4634305.0,9523833.0,37.890650,-122.318116,"POLYGON ((-122.373121 37.88388399999999, -122....",0,3612,647,742,0.205426,0.179125,8680.105205,61127.0,african_american,-41.4,-23.1,-18.3
2,Alameda,Albany City Unified,0.2833,"29,644,972","1,679,684",-,31352540,1611270000000,6.0,1860.0,601860.0,4634305.0,9523833.0,37.890650,-122.318116,"POLYGON ((-122.373121 37.88388399999999, -122....",0,3612,647,742,0.205426,0.179125,8680.105205,61127.0,american_indian,NaN,NaN,NaN
3,Alameda,Albany City Unified,0.2833,"29,644,972","1,679,684",-,31352540,1611270000000,6.0,1860.0,601860.0,4634305.0,9523833.0,37.890650,-122.318116,"POLYGON ((-122.373121 37.88388399999999, -122....",0,3612,647,742,0.205426,0.179125,8680.105205,61127.0,all_students,41.2,48.1,-6.9
4,Alameda,Albany City Unified,0.2833,"29,644,972","1,679,684",-,31352540,1611270000000,6.0,1860.0,601860.0,4634305.0,9523833.0,37.890650,-122.318116,"POLYGON ((-122.373121 37.88388399999999, -122....",0,3612,647,742,0.205426,0.179125,8680.105205,61127.0,asian,76.2,74.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13344,Yuba,Wheatland Union High,0.4283,"7,062,422","604,967",-,7852628,58727690000000,6.0,NaN,642350.0,400896712.0,3658509.0,39.085259,-121.407370,"POLYGON ((-121.589044 38.991522, -121.588379 3...",0,765,253,335,0.437908,0.330719,10264.873203,72769.0,pacific_islander,NaN,NaN,NaN
13345,Yuba,Wheatland Union High,0.4283,"7,062,422","604,967",-,7852628,58727690000000,6.0,NaN,642350.0,400896712.0,3658509.0,39.085259,-121.407370,"POLYGON ((-121.589044 38.991522, -121.588379 3...",0,765,253,335,0.437908,0.330719,10264.873203,72769.0,rfep_only,NaN,NaN,NaN
13346,Yuba,Wheatland Union High,0.4283,"7,062,422","604,967",-,7852628,58727690000000,6.0,NaN,642350.0,400896712.0,3658509.0,39.085259,-121.407370,"POLYGON ((-121.589044 38.991522, -121.588379 3...",0,765,253,335,0.437908,0.330719,10264.873203,72769.0,ses_disad,-90.0,-74.0,-16.0
13347,Yuba,Wheatland Union High,0.4283,"7,062,422","604,967",-,7852628,58727690000000,6.0,NaN,642350.0,400896712.0,3658509.0,39.085259,-121.407370,"POLYGON ((-121.589044 38.991522, -121.588379 3...",0,765,253,335,0.437908,0.330719,10264.873203,72769.0,disabilities,-226.6,NaN,NaN


### RDD
need to query counties to see if they contain school districts that have certain properties. 

e.g.: one county that contains both:
* districts with very high (> 80%) upc
* districts with very low (< 20%) upc

also more narrorw breakpoints
e.g. one county that countains both: 
* districts with 40-54% upc
* districts with 55-60% upc

hopefully will find multiple counties that meet both criteria to compare.

In [8]:
for county in df.county_name:
    print(county)

AttributeError: 'DataFrame' object has no attribute 'county_name'

## pair treatment/non-treatment by...

geography: neighbors or same county

demographics : upc or census make up

income

In [ ]:
import geosnap

In [ ]:
gdf = gpd.GeoDataFrame(df)